This is transformer implementation of the model

In [18]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np


We assume thte data is loeaded and preprocessed and df is the dataframe
we ignore the data loading and preprocessing steps becuase it is same with the traditional implementation

In [19]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)


label_encoder = LabelEncoder()
df['emotion_encoded'] = label_encoder.fit_transform(df['emotion'])


X_train, X_test, y_train, y_test = train_test_split(df['text_processed'], df['emotion_encoded'], test_size=0.2, random_state=42, stratify=df['emotion_encoded'])

train_encodings = tokenizer(list(X_train), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(X_test), padding=True, truncation=True, max_length=128)


In [23]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)


label_encoder = LabelEncoder()
df['emotion_encoded'] = label_encoder.fit_transform(df['emotion'])


X_train, X_test, y_train, y_test = train_test_split(df['text_processed'], df['emotion_encoded'], test_size=0.2, random_state=42, stratify=df['emotion_encoded'])

train_encodings = tokenizer(list(X_train), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(X_test), padding=True, truncation=True, max_length=128)

# Create a dataset class for BERT
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = EmotionDataset(train_encodings, y_train.values)
test_dataset = EmotionDataset(test_encodings, y_test.values)


model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # Updated argument
    save_strategy="epoch",
    report_to="none"  # Disable WandB logging
)

# Define the Trainer and train the model
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=lambda p: {
        'accuracy': (np.array(p.predictions).argmax(axis=1) == np.array(p.label_ids)).mean()
    }
)


trainer.train()


predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)


print(classification_report(y_test, preds, target_names=label_encoder.classes_))

# Save the model and tokenizer
model.save_pretrained('./emotion_bert_model')
tokenizer.save_pretrained('./emotion_bert_model')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.697800,0.819311,0.710378
2,0.809700,0.759706,0.734914
3,0.505400,0.808700,0.733588


              precision    recall  f1-score   support

       anger       0.48      0.31      0.38       113
     disgust       1.00      0.12      0.22         8
        fear       0.44      0.29      0.35        56
         joy       0.69      0.62      0.65       548
     neutral       0.80      0.87      0.83      3676
     sadness       0.61      0.62      0.62      1123
    surprise       0.44      0.28      0.34       508

    accuracy                           0.73      6032
   macro avg       0.64      0.44      0.49      6032
weighted avg       0.72      0.73      0.72      6032



('./emotion_bert_model/tokenizer_config.json',
 './emotion_bert_model/special_tokens_map.json',
 './emotion_bert_model/vocab.txt',
 './emotion_bert_model/added_tokens.json')

In [24]:
import pickle

# Save the model and tokenizer to pickle files
with open('emotion_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('emotion_tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)


In [25]:
# Save the model using torch
torch.save(model.state_dict(), 'emotion_model.pth')

# Save the tokenizer using pickle (it's easier to load later)
with open('emotion_tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)
